## Import libraries

In [1]:
import pandas as pd 
import itertools 
import cvxpy as cp
import numpy as np
import math
from pypfopt.risk_models import sample_cov, CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return
from pypfopt import EfficientFrontier

import warnings
warnings.filterwarnings("ignore")

## Read the data from csv file

In [2]:
data = pd.read_csv("Data.csv")

### Explore the data and rename columns

In [3]:
data.head()

,<Ticker>,<DTYYYYMMDD>,<Open>,<High>,<Low>,<Close>,<Volume>
0,VNAll-INDEX,20220923,1204.92,1208.80,1192.55,1194.23,363201500
1,AAA,20220923,10.95,10.95,10.70,10.70,1284600
2,AAM,20220923,12.10,12.10,11.90,12.10,3300
3,AAT,20220923,10.80,10.80,10.40,10.70,1475100
4,ABR,20220923,11.75,11.75,11.75,11.75,100


- BMP, DHG, FPT, GAS, HAG, HCM, IMP, MSN, VCB, VIC

## Rename the columns

In [4]:
columns = ["Ticker", "YYYYMMDD", "Open", "High", "Low", "Close", "Volume"]
data.columns = columns

## Read the data again for check

In [5]:
data.head()

,Ticker,YYYYMMDD,Open,High,Low,Close,Volume
0,VNAll-INDEX,20220923,1204.92,1208.80,1192.55,1194.23,363201500
1,AAA,20220923,10.95,10.95,10.70,10.70,1284600
2,AAM,20220923,12.10,12.10,11.90,12.10,3300
3,AAT,20220923,10.80,10.80,10.40,10.70,1475100
4,ABR,20220923,11.75,11.75,11.75,11.75,100


In [6]:
data.tail()

,Ticker,YYYYMMDD,Open,High,Low,Close,Volume
1026411,SAM,20000802,2.1788,2.1788,2.1788,2.1803,200
1026412,REE,20000731,1.3774,1.3774,1.3774,1.3784,300
1026413,SAM,20000731,2.1414,2.1414,2.1414,2.1429,10000
1026414,REE,20000728,1.3520,1.3520,1.3520,1.3530,1000
1026415,SAM,20000728,2.1165,2.1165,2.1165,2.1180,3200


In [7]:
data["Ticker"].describe()

count     1026416
unique       1182
top           REE
freq         5368
Name: Ticker, dtype: object

## Take out the ten ticker that we need

In [8]:
top_ten = ["BMP", "DHG", "FPT", "GAS", "HAG", "HCM", "IMP", "MSN", "VCB", "VIC"]
data_10 = data[data["Ticker"].isin(top_ten)]
data_10.reset_index(inplace = True)
data_10 = data_10.drop("index", axis = 1)

data_10.head()

,Ticker,YYYYMMDD,Open,High,Low,Close,Volume
0,BMP,20220923,60.3,60.4,59.7,59.9,86900
1,DHG,20220923,88.5,89.8,88.5,89.0,1900
2,FPT,20220923,83.1,83.1,82.1,82.4,840600
3,GAS,20220923,111.4,114.5,111.0,112.5,361700
4,HAG,20220923,14.0,14.6,13.9,13.9,38855700


#### Check the number of tickers again

In [9]:
data_10["Ticker"].unique() # doc nhat 

array(['BMP', 'DHG', 'FPT', 'GAS', 'HAG', 'HCM', 'IMP', 'MSN', 'VCB',
       'VIC'], dtype=object)

## Take out the tickers from the year 2016 to 2022

In [10]:
data_10['YYYYMMDD'] = data_10['YYYYMMDD'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
year = [i for i in range(2016, 2023)]

result = data_10[(data_10["YYYYMMDD"].dt.year.isin(year))] 

result.head()

,Ticker,YYYYMMDD,Open,High,Low,Close,Volume
0,BMP,2022-09-23,60.3,60.4,59.7,59.9,86900
1,DHG,2022-09-23,88.5,89.8,88.5,89.0,1900
2,FPT,2022-09-23,83.1,83.1,82.1,82.4,840600
3,GAS,2022-09-23,111.4,114.5,111.0,112.5,361700
4,HAG,2022-09-23,14.0,14.6,13.9,13.9,38855700


## Just copying the data to prevent us from losing data during proccessing 

In [11]:
pre = result.copy()
pre["day"] = pre["YYYYMMDD"].dt.day
pre["month"] = pre["YYYYMMDD"].dt.month
pre["year"] = pre["YYYYMMDD"].dt.year

In [12]:
pre = pre.drop(columns = ["Close"])

## Take out the First Notice Day of each month (from 2016 to 2022) for tickers

In [13]:
first_day = pre.groupby(by =  ["Ticker", "year","month"]).agg('min')

first_day = first_day.reset_index(col_level = [1, 2])
first_day = first_day.drop(["year", "month", "day", "Open", "High", "Low", "Volume"], axis = 1)

indexMonth = first_day[(first_day['YYYYMMDD'].dt.year == 2016) & (first_day['YYYYMMDD'].dt.month <= 8) ].index
indexMonth2 = first_day[(first_day['YYYYMMDD'].dt.year == 2022) & (first_day['YYYYMMDD'].dt.month == 9) ].index

first_day = first_day.drop(indexMonth)
first_day = first_day.drop(indexMonth2)

In [14]:
first_day = pd.merge(first_day, result, on = ["YYYYMMDD", "Ticker"])

In [15]:
first_day.head()

,Ticker,YYYYMMDD,Open,High,Low,Close,Volume
0,BMP,2016-09-01,59.7452,59.7452,59.0166,59.7492,74850
1,BMP,2016-10-03,69.9456,71.7307,69.9456,71.4076,211220
2,BMP,2016-11-01,73.2243,73.4793,71.4028,73.4842,74390
3,BMP,2016-12-01,70.1278,71.0021,70.1278,70.8611,7600
4,BMP,2017-01-03,69.9390,69.9390,69.5709,69.9491,17680


## Just take the close value

In [16]:
first_day = first_day.drop(labels = ["Open", "High", "Low", "Volume"], axis = 1)
first_day.head()

,Ticker,YYYYMMDD,Close
0,BMP,2016-09-01,59.7492
1,BMP,2016-10-03,71.4076
2,BMP,2016-11-01,73.4842
3,BMP,2016-12-01,70.8611
4,BMP,2017-01-03,69.9491


In [17]:
test = first_day.copy()

In [18]:
first_day["Return"] = first_day.groupby("Ticker").Close.pct_change()

In [19]:
first_day.head()

,Ticker,YYYYMMDD,Close,Return
0,BMP,2016-09-01,59.7492,NaN
1,BMP,2016-10-03,71.4076,0.195122
2,BMP,2016-11-01,73.4842,0.029081
3,BMP,2016-12-01,70.8611,-0.035696
4,BMP,2017-01-03,69.9491,-0.012870


In [20]:
first_day.tail()

,Ticker,YYYYMMDD,Close,Return
715,VIC,2022-04-01,82.1,0.036616
716,VIC,2022-05-04,79.7,-0.029233
717,VIC,2022-06-01,79.3,-0.005019
718,VIC,2022-07-01,72.6,-0.084489
719,VIC,2022-08-01,64.7,-0.108815


## Mean return of each ticker

In [21]:
mean_data = first_day.groupby("Ticker").agg({"Return": "mean"})
mean_data.columns = ["Mean"]
mean_data.T

Ticker,BMP,DHG,FPT,GAS,HAG,HCM,IMP,MSN,VCB,VIC
Mean,0.005607,0.012172,0.026866,0.017869,0.020997,0.02798,0.010882,0.021974,0.017839,0.014884


## Var data

In [22]:
var_data =  first_day.groupby("Ticker")["Return"].var(ddof = 0)

In [23]:
var_data = var_data.to_frame()
var_data.columns = ["Var"]

In [24]:
var_data.T

Ticker,BMP,DHG,FPT,GAS,HAG,HCM,IMP,MSN,VCB,VIC
Var,0.010733,0.009444,0.006148,0.012421,0.026707,0.024396,0.006431,0.01228,0.007063,0.007156


## Standard Deviation 

In [25]:
std_data = first_day.groupby("Ticker")["Return"].std(ddof = 0)
std_data = std_data.to_frame()
std_data.columns = ["Std"]

In [26]:
std_data.T

Ticker,BMP,DHG,FPT,GAS,HAG,HCM,IMP,MSN,VCB,VIC
Std,0.103599,0.097182,0.078412,0.11145,0.163422,0.156193,0.080196,0.110816,0.084042,0.084594


In [27]:
df4 = pd.merge(pd.merge(mean_data,var_data,on='Ticker'), std_data, on='Ticker')
df4.head()

,Mean,Var,Std
Ticker,,,
BMP,0.005607,0.010733,0.103599
DHG,0.012172,0.009444,0.097182
FPT,0.026866,0.006148,0.078412
GAS,0.017869,0.012421,0.111450
HAG,0.020997,0.026707,0.163422


## Covariance and correlation of Return

In [28]:
corr_table = first_day.set_index(['Ticker', 'YYYYMMDD']).Return.unstack(['Ticker']).corr()
corr_table

Ticker,BMP,DHG,FPT,GAS,HAG,HCM,IMP,MSN,VCB,VIC
Ticker,,,,,,,,,,
BMP,1.000000,0.454541,0.384134,0.357854,0.180886,0.469396,0.375093,0.134963,0.412359,0.131881
DHG,0.454541,1.000000,0.260682,0.232184,0.246093,0.458201,0.351418,0.233302,0.242383,0.255163
FPT,0.384134,0.260682,1.000000,0.510033,0.230500,0.545507,0.261883,0.328088,0.417735,0.274268
GAS,0.357854,0.232184,0.510033,1.000000,0.161148,0.528133,0.332331,0.400105,0.637657,0.340277
HAG,0.180886,0.246093,0.230500,0.161148,1.000000,0.453296,0.243918,0.238688,0.340661,0.195149
HCM,0.469396,0.458201,0.545507,0.528133,0.453296,1.000000,0.463999,0.397924,0.601752,0.316153
IMP,0.375093,0.351418,0.261883,0.332331,0.243918,0.463999,1.000000,0.151163,0.344321,0.260494
MSN,0.134963,0.233302,0.328088,0.400105,0.238688,0.397924,0.151163,1.000000,0.259819,0.403503
VCB,0.412359,0.242383,0.417735,0.637657,0.340661,0.601752,0.344321,0.259819,1.000000,0.437601


In [29]:
first_day.fillna(0, inplace = True)
cov_table = first_day.set_index(['YYYYMMDD', "Ticker"])#.Return.to_frame()
df1 = cov_table.Return.unstack("Ticker")
covar = df1.cov(ddof = 1)
covar

Ticker,BMP,DHG,FPT,GAS,HAG,HCM,IMP,MSN,VCB,VIC
Ticker,,,,,,,,,,
BMP,0.010733,0.004588,0.003144,0.004133,0.003064,0.007598,0.003117,0.001542,0.003562,0.001164
DHG,0.004588,0.009446,0.001991,0.002498,0.003947,0.006969,0.002767,0.002516,0.001983,0.002100
FPT,0.003144,0.001991,0.006158,0.004448,0.003001,0.006729,0.001674,0.002859,0.002759,0.001825
GAS,0.004133,0.002498,0.004448,0.012425,0.002940,0.009200,0.002973,0.004868,0.005868,0.003197
HAG,0.003064,0.003947,0.003001,0.002940,0.026713,0.011579,0.003200,0.004331,0.004675,0.002735
HCM,0.007598,0.006969,0.006729,0.009200,0.011579,0.024407,0.005816,0.006848,0.007832,0.004202
IMP,0.003117,0.002767,0.001674,0.002973,0.003200,0.005816,0.006433,0.001348,0.002321,0.001792
MSN,0.001542,0.002516,0.002859,0.004868,0.004331,0.006848,0.001348,0.012287,0.002425,0.003787
VCB,0.003562,0.001983,0.002759,0.005868,0.004675,0.007832,0.002321,0.002425,0.007067,0.003115


## Sharpe raito maximization

In [30]:
raito = pd.concat([mean_data.T, std_data.T], ignore_index = False)
raito = raito.rename_axis(None, axis=1)

rf = [20/1200, 0]
raito.insert(loc = 0, column='rf', value = rf)

raito

,rf,BMP,DHG,FPT,GAS,HAG,HCM,IMP,MSN,VCB,VIC
Mean,0.016667,0.005607,0.012172,0.026866,0.017869,0.020997,0.027980,0.010882,0.021974,0.017839,0.014884
Std,0.000000,0.103599,0.097182,0.078412,0.111450,0.163422,0.156193,0.080196,0.110816,0.084042,0.084594


In [31]:
ticker = list(result["Ticker"].unique())
combination = list(itertools.combinations(ticker, 3))

In [32]:
df = pd.DataFrame({"Combination":combination})
df["Combination"] = df["Combination"].apply(lambda x: '-'.join(x))

In [33]:
df["wi"] = 1/3
df["wj"] = 1/3 
df["wk"] = 1/3

df["Total"] = df["wi"] + df["wj"] + df["wk"]

In [34]:
df["Stock i"] = df["Combination"].apply(lambda x: x.split('-')[0])
df["Stock j"] = df["Combination"].apply(lambda x: x.split('-')[1])
df["Stock k"] = df["Combination"].apply(lambda x: x.split('-')[2])

In [35]:
# df["E(ri)"] = raito.loc["Mean"][df["Stock i"]].values
# df["E(rj)"] = raito.loc["Mean"][df["Stock j"]].values
# df["E(rk)"] = raito.loc["Mean"][df["Stock k"]].values

In [36]:
# df["𝞼(ri)"] = raito.loc["Std"][df["Stock i"]].values
# df["𝞼(rj)"] = raito.loc["Std"][df["Stock j"]].values
# df["𝞼(rk)"] = raito.loc["Std"][df["Stock k"]].values

In [37]:
# df["Cov(i,j)"] = [covar[i][j] for i, j in zip(df["Stock i"], df["Stock j"])]
# df["Cov(i,k)"] = [covar[i][k] for i, k in zip(df["Stock i"], df["Stock k"])]
# df["Cov(j,k)"] = [covar[j][k] for j, k in zip(df["Stock j"], df["Stock k"])]

In [38]:
# df["E(rp)"] = df["wi"] * df["E(ri)"] + df["wj"] * df["E(rj)"] + df["wk"] * df["E(rk)"]

# df["𝞼p"] = np.sqrt(  (df["wi"] ** 2) * (df["𝞼(ri)"] ** 2) 
#                     +(df["wj"] ** 2) * (df["𝞼(rj)"] ** 2)
#                     +(df["wk"] ** 2) * (df["𝞼(rk)"] ** 2) 
#                     + 2 * df["wi"] * df["wj"] * df["Cov(i,j)"]
#                     + 2 * df["wi"] * df["wk"] * df["Cov(i,k)"]
#                     + 2 * df["wj"] * df["wk"] * df["Cov(j,k)"]  )

# df["Sharpe ratio"] = (df["E(rp)"] - raito["rf"]["Mean"]) / df["𝞼p"]

In [39]:
df.head()

,Combination,wi,wj,wk,Total,Stock i,Stock j,Stock k
0,BMP-DHG-FPT,0.333333,0.333333,0.333333,1.0,BMP,DHG,FPT
1,BMP-DHG-GAS,0.333333,0.333333,0.333333,1.0,BMP,DHG,GAS
2,BMP-DHG-HAG,0.333333,0.333333,0.333333,1.0,BMP,DHG,HAG
3,BMP-DHG-HCM,0.333333,0.333333,0.333333,1.0,BMP,DHG,HCM
4,BMP-DHG-IMP,0.333333,0.333333,0.333333,1.0,BMP,DHG,IMP


In [40]:
df.tail()

,Combination,wi,wj,wk,Total,Stock i,Stock j,Stock k
115,HCM-VCB-VIC,0.333333,0.333333,0.333333,1.0,HCM,VCB,VIC
116,IMP-MSN-VCB,0.333333,0.333333,0.333333,1.0,IMP,MSN,VCB
117,IMP-MSN-VIC,0.333333,0.333333,0.333333,1.0,IMP,MSN,VIC
118,IMP-VCB-VIC,0.333333,0.333333,0.333333,1.0,IMP,VCB,VIC
119,MSN-VCB-VIC,0.333333,0.333333,0.333333,1.0,MSN,VCB,VIC


### Optimization

In [44]:
for i in range(0, 120):
    close = [df["Stock i"][i], df["Stock j"][i], df["Stock k"][i]]
    opti = test.set_index(['YYYYMMDD', "Ticker"]).Close.unstack("Ticker")[close]


    mu = mean_historical_return(opti, frequency=12)
    S = CovarianceShrinkage(opti,frequency=12).ledoit_wolf()
    #S = sample_cov(opti, frequency=13)

    ef = EfficientFrontier(mu, S, weight_bounds=(0.05, 1),)
    ef.max_sharpe(risk_free_rate = 20/1200)
    weights = ef.clean_weights()
    
    weights_df = pd.DataFrame.from_dict(weights, orient = 'index')
    weights_df.columns = ['weights']
    
    
    df["wi"][i] = weights_df.values.T[0][0]
    df["wj"][i] = weights_df.values.T[0][1]
    df["wk"][i] = weights_df.values.T[0][2]

In [45]:
df.head()

,Combination,wi,wj,wk,Total,Stock i,Stock j,Stock k
0,BMP-DHG-FPT,0.05,0.05000,0.90000,1.0,BMP,DHG,FPT
1,BMP-DHG-GAS,0.05,0.35409,0.59591,1.0,BMP,DHG,GAS
2,BMP-DHG-HAG,0.05,0.62687,0.32313,1.0,BMP,DHG,HAG
3,BMP-DHG-HCM,0.05,0.26288,0.68712,1.0,BMP,DHG,HCM
4,BMP-DHG-IMP,0.05,0.39158,0.55842,1.0,BMP,DHG,IMP


In [46]:
df.tail()

,Combination,wi,wj,wk,Total,Stock i,Stock j,Stock k
115,HCM-VCB-VIC,0.06760,0.59076,0.34165,1.0,HCM,VCB,VIC
116,IMP-MSN-VCB,0.15666,0.40787,0.43547,1.0,IMP,MSN,VCB
117,IMP-MSN-VIC,0.21533,0.45743,0.32723,1.0,IMP,MSN,VIC
118,IMP-VCB-VIC,0.14788,0.53042,0.32170,1.0,IMP,VCB,VIC
119,MSN-VCB-VIC,0.35589,0.43561,0.20850,1.0,MSN,VCB,VIC


In [47]:
df.

SyntaxError: invalid syntax (791285630.py, line 1)